<a href="https://colab.research.google.com/github/Karthikbv29/Batch108/blob/main/Tennis_pipeline_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('tennis.csv')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
data.shape

(8001, 27)

In [ ]:
data.head()

,rally,serve,hitpoint,speed,net.clearance,distance.from.sideline,depth,outside.sideline,outside.baseline,player.distance.travelled,...,previous.depth,opponent.depth,opponent.distance.from.center,same.side,previous.hitpoint,previous.time.to.net,server.is.impact.player,outcome,gender,ID
0,4,1,B,35.515042,-0.021725,3.474766,6.797621,False,False,1.467570,...,0.705435,12.5628,2.0724,True,F,0.445318,False,UE,mens,8644
1,4,2,B,33.382640,1.114202,2.540801,2.608708,False,True,2.311931,...,3.856600,12.3544,5.1124,False,B,0.432434,False,FE,mens,1182
2,23,1,B,22.316690,-0.254046,3.533166,9.435749,False,False,3.903728,...,2.908892,13.8620,1.6564,False,F,0.397538,True,FE,mens,9042
3,9,1,F,36.837309,0.766694,0.586885,3.342180,True,False,0.583745,...,0.557554,14.2596,0.1606,True,B,0.671984,True,UE,mens,1222
4,4,1,B,35.544208,0.116162,0.918725,5.499119,False,False,2.333456,...,3.945317,11.3658,1.1082,False,F,0.340411,False,W,mens,4085


In [ ]:
data.drop(['ID'],axis=1,inplace=True)

In [ ]:
data.dtypes

rally                                   int64
serve                                   int64
hitpoint                               object
speed                                 float64
net.clearance                         float64
distance.from.sideline                float64
depth                                 float64
outside.sideline                         bool
outside.baseline                         bool
player.distance.travelled             float64
player.impact.depth                   float64
player.impact.distance.from.center    float64
player.depth                          float64
player.distance.from.center           float64
previous.speed                        float64
previous.net.clearance                float64
previous.distance.from.sideline       float64
previous.depth                        float64
opponent.depth                        float64
opponent.distance.from.center         float64
same.side                                bool
previous.hitpoint                 

In [ ]:
data.drop_duplicates(keep='first',inplace=True)
data.shape

(8001, 26)

In [ ]:
data.nunique()

rally                                   30
serve                                    2
hitpoint                                 4
speed                                 8001
net.clearance                         7994
distance.from.sideline                8001
depth                                 8001
outside.sideline                         2
outside.baseline                         2
player.distance.travelled             7988
player.impact.depth                   7089
player.impact.distance.from.center    6644
player.depth                          6882
player.distance.from.center           5977
previous.speed                        8001
previous.net.clearance                8001
previous.distance.from.sideline       8001
previous.depth                        8001
opponent.depth                        6856
opponent.distance.from.center         6775
same.side                                2
previous.hitpoint                        4
previous.time.to.net                  8001
server.is.i

In [ ]:
unique=['outcome','speed','distance.from.sideline','depth','previous.speed','previous.net.clearance','previous.distance.from.sideline','previous.depth','previous.time.to.net']

In [ ]:
x=data.drop(['outcome','speed','distance.from.sideline','depth','previous.speed','previous.net.clearance','previous.distance.from.sideline','previous.depth','previous.time.to.net'],axis=1)
y=data['outcome']

In [ ]:
x.shape

(8001, 17)

In [ ]:
num_cols=x.select_dtypes(include=['int64','float64']).columns
cat_cols=x.select_dtypes(include=['bool','object','category']).columns

In [ ]:
num_cols

Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distance.from.center'],
      dtype='object')

In [ ]:
cat_cols

Index(['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side',
       'previous.hitpoint', 'server.is.impact.player', 'gender'],
      dtype='object')

In [ ]:
for col in cat_cols:
  data[col]=data[col].astype('category')

In [ ]:
y=y.astype('category')


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=123)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5600, 17)
(2401, 17)
(5600,)
(2401,)


In [ ]:
y.value_counts()

UE    3501
W     2682
FE    1818
Name: outcome, dtype: int64

In [ ]:
def evaluate_model(act, pred):
    from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred,average='weighted'))
    print("Precision: ", precision_score(act, pred,average='weighted'))    

In [ ]:
numeric_transformer = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]
)

In [ ]:
categorical_tranformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_tranformer, cat_cols)
    ]
)

In [ ]:
RF_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

In [ ]:
RF_pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distance.from.center'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
            

In [ ]:
train_pred_RFPipe = RF_pipe.predict(x_train)
train_pred_RFPipe

array(['FE', 'FE', 'UE', ..., 'W', 'W', 'UE'], dtype=object)

In [ ]:
test_pred_RFPipe = RF_pipe.predict(x_test)
test_pred_RFPipe

array(['W', 'W', 'FE', ..., 'UE', 'UE', 'FE'], dtype=object)

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_RFPipe)
print("--Test--")
evaluate_model(y_test, test_pred_RFPipe)

--Train--
Confusion Matrix 
 [[1300    0    0]
 [   0 2419    0]
 [   0    0 1881]]
Accurcay :  1.0
Recall   :  1.0
Precision:  1.0
--Test--
Confusion Matrix 
 [[311 184  23]
 [121 898  63]
 [ 12  23 766]]
Accurcay :  0.8225739275301958
Recall   :  0.8225739275301958
Precision:  0.8172802240492898


In [ ]:
from sklearn import set_config
set_config (display='diagram')
RF_pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distance.from.center'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side',
       'previous.hitpoint', 'server.is.impact.player', 'gender'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [ ]:
Adaboost_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', AdaBoostClassifier())])

In [ ]:
Adaboost_pipe.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distance.from.center'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side',
       'previous.hitpoint', 'server.is.impact.player', 'gender'],
      dtype='object'))])),
                ('classifier', AdaBoostClassifier())])

In [ ]:
train_pred_adapipe = Adaboost_pipe.predict(x_train)
test_pred_adapipe = Adaboost_pipe.predict(x_test)

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_adapipe)
print("--Test--")
evaluate_model(y_test, test_pred_adapipe)

--Train--
Confusion Matrix 
 [[ 587  647   66]
 [ 188 2140   91]
 [  49   64 1768]]
Accurcay :  0.8026785714285715
Recall   :  0.8026785714285715
Precision:  0.798109934700452
--Test--
Confusion Matrix 
 [[236 258  24]
 [104 934  44]
 [ 27  42 732]]
Accurcay :  0.7921699291961682
Recall   :  0.7921699291961682
Precision:  0.7850765580249673


In [ ]:
Gradientboost_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', GradientBoostingClassifier())])

In [ ]:
Gradientboost_pipe.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distance.from.center'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side',
       'previous.hitpoint', 'server.is.impact.player', 'gender'],
      dtype='object'))])),
                ('classifier', GradientBoostingClassifier())])

In [ ]:
train_pred_Grapipe = Gradientboost_pipe.predict(x_train)
test_pred_Grapipe = Gradientboost_pipe.predict(x_test)

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_Grapipe)
print("--Test--")
evaluate_model(y_test, test_pred_Grapipe)

--Train--
Confusion Matrix 
 [[ 889  364   47]
 [ 164 2174   81]
 [  18   33 1830]]
Accurcay :  0.87375
Recall   :  0.87375
Precision:  0.8718910175238749
--Test--
Confusion Matrix 
 [[305 194  19]
 [127 906  49]
 [ 13  24 764]]
Accurcay :  0.8225739275301958
Recall   :  0.8225739275301958
Precision:  0.8174565885816877


In [ ]:
XGBoost_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', XGBClassifier())])

In [ ]:
XGBoost_pipe.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rally', 'serve', 'net.clearance', 'player.distance.travelled',
       'player.impact.depth', 'player.impact.distance.from.center',
       'player.depth', 'player.distance.from.center', 'opponent.depth',
       'opponent.distanc...r'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side',
       'previous.hitpoint', 'server.is.impact.player', 'gender'],
      dtype='object'))])),
                ('classifier', XGBClassifier(objective='multi:softprob'))])

In [ ]:
train_pred_xgbpipe = XGBoost_pipe.predict(x_train)
test_pred_xgbpipe = XGBoost_pipe.predict(x_test)

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_xgbpipe)
print("--Test--")
evaluate_model(y_test, test_pred_xgbpipe)

--Train--
Confusion Matrix 
 [[ 848  398   54]
 [ 170 2165   84]
 [  22   31 1828]]
Accurcay :  0.8644642857142857
Recall   :  0.8644642857142857
Precision:  0.8621264675073786
--Test--
Confusion Matrix 
 [[305 193  20]
 [120 911  51]
 [ 11  26 764]]
Accurcay :  0.8246563931695127
Recall   :  0.8246563931695127
Precision:  0.8194735769064679


In [ ]:
performance_columns = ['Model name', 'Train accuracy', 'Train precision', 'Train recall','Test accuracy', 'Test precision', 'Test recall']
performance_comparison = pd.DataFrame(columns=performance_columns)

In [ ]:
def add_to_perform_compare_df(df, model_name, train_actual, train_predict, test_actual, test_predict):
    
    from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
    
    train_accuracy = accuracy_score(train_actual, train_predict)
    test_accuracy = accuracy_score(test_actual, test_predict)
    
    train_recall = recall_score(train_actual, train_predict,average='weighted')
    test_recall = recall_score(test_actual,test_predict,average='weighted')
    
    train_precision = precision_score(train_actual, train_predict,average='weighted')
    test_precision = precision_score(test_actual, test_predict,average='weighted')
    
    df = df.append(pd.Series([model_name, train_accuracy, train_precision, train_recall,
                              test_accuracy, test_precision, test_recall],index=df.columns),ignore_index=True)
    return df

In [ ]:
performance_comparison = add_to_perform_compare_df(performance_comparison, 'Random Forest', y_train, train_pred_RFPipe, y_test, test_pred_RFPipe)

In [ ]:
performance_comparison = add_to_perform_compare_df(performance_comparison, 'Ada Boost', y_train, train_pred_adapipe, y_test, test_pred_adapipe)

In [ ]:
performance_comparison = add_to_perform_compare_df(performance_comparison, 'Gradient Boost', y_train, train_pred_Grapipe, y_test, test_pred_Grapipe)

In [ ]:
performance_comparison = add_to_perform_compare_df(performance_comparison, 'XG Boost', y_train, train_pred_xgbpipe, y_test, test_pred_xgbpipe)

In [ ]:
performance_comparison

,Model name,Train accuracy,Train precision,Train recall,Test accuracy,Test precision,Test recall
0,Random Forest,1.000000,1.000000,1.000000,0.822574,0.817280,0.822574
1,Ada Boost,0.802679,0.798110,0.802679,0.792170,0.785077,0.792170
2,Gradient Boost,0.873750,0.871891,0.873750,0.822574,0.817457,0.822574
3,XG Boost,0.864464,0.862126,0.864464,0.824656,0.819474,0.824656
